In [42]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import requests
import time
import Constants as CONST
import hashlib
import hmac

In [2]:
df = pd.read_csv('bnb_historical_data.csv')

df_24 = pd.read_csv('bnb_historical_data_24.csv')
combined_data = pd.concat([df, df_24], axis=0, ignore_index=True)

In [3]:
def calculate_moving_averages(df, window_sizes):
    for window_size in window_sizes:
        column_name = f'MA_{window_size}'
        df[column_name] = df['close'].rolling(window=window_size).mean()
    return df

In [4]:
window_sizes = [7, 25, 99]
combined_data = calculate_moving_averages(combined_data, window_sizes)

In [8]:
desired_months = [1, 2, 3]  # January, February, and March
combined_data['timestamp'] = pd.to_datetime(combined_data['timestamp']) 
filtered_df = combined_data[combined_data['timestamp'].dt.month.isin(desired_months)]

In [10]:
candlestick_trace  = go.Candlestick(x=df['timestamp'],
                open=filtered_df['open'],
                high=filtered_df['high'],
                low=filtered_df['low'],
                close=filtered_df['close'])

symbol = 'BNBUSDT'

trace_ma_7 = go.Scatter(x=filtered_df['timestamp'], y=filtered_df['MA_7'], mode='lines', name='MA(7)', line=dict(color='green', dash='dash'))
trace_ma_25 = go.Scatter(x=filtered_df['timestamp'], y=filtered_df['MA_25'], mode='lines', name='MA(25)', line=dict(color='red', dash='dash'))
trace_ma_99 = go.Scatter(x=filtered_df['timestamp'], y=filtered_df['MA_99'], mode='lines', name='MA(99)', line=dict(color='purple', dash='dash'))



fig = go.Figure(data=[candlestick_trace, trace_ma_7, trace_ma_25, trace_ma_99])

fig.update_layout(title=f'Historical Price Data for {symbol}',
                  xaxis_title='Timestamp',
                  yaxis_title='Price (USDT)',
                  xaxis_rangeslider_visible=False)

fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
class Account:
    
    def __init__(self, start_money):
        self.money = start_money
        self.bnbvalue = 0
        self.holding = False
    
    def Buy(self, Price):
        self.bnbvalue = self.money / Price
        self.holding = True
        self.money = 0
        
    def Sell(self, Price):
        self.money = self.bnbvalue * Price
        self.holding = False
        self.bnbvalue = 0
        

In [38]:
test_acc = Account(100.)

In [39]:
for index, row in filtered_df.iterrows():
    #print(f"{index} -- MA(7) {row['MA_7']}, MA(25) {row['MA_25']}")
    
    if(index > 25):
        
        if(row['MA_7'] > row['MA_25']):
            if test_acc.holding:
                test_acc.Sell(row['close'])
        else:
            if not test_acc.holding:
                test_acc.Buy(row['close'])

if(test_acc.holding):
    test_acc.Sell(row['close'])
    
print(test_acc.money)

Money 99.9187652315191
Money 99.95941566489238
Money 99.95941566489238
Money 100.04081584051525
Money 100.08158226913812
Money 100.04079922827866
Money 100.08158226913812
Money 100.12239857512066
Money 99.6322032332596
Money 99.6322032332596
Money 99.71390245690311
Money 99.75475206872487
Money 99.75475206872487
Money 99.79565192607161
Money 99.79565192607161
Money 99.87745164076512
Money 99.87745164076512
Money 99.83648467946865
Money 99.79561833289497
Money 99.79561833289497
Money 99.83646795153308
Money 99.91823410055154
Money 99.9591004539669
Money 100.08185006565822
Money 100.04090004517309
Money 100.08186683470839
Money 100.12285040016077
Money 100.1638507484
Money 100.20486788629862
Money 100.45107149289888
Money 100.49212219641049
Money 100.53320646550142
Money 100.53320646550142
Money 100.57420695753467
Money 100.57420695753467
Money 100.49217253097878
Money 100.45123885581341
Money 100.41032185424282
Money 100.41032185424282
Money 100.45127223020296
Money 100.28740392313901
M

In [48]:
class BinanceAccount:
    
    def __inti__(self, symbol='BNBUSDT', interval='1m'):
        self.symbol = symbol
        self.base_url = 'https://api.binance.com/api/v3'
        self.df = pd.DataFrame()
        self.interval = interval
        
    def place_order(self, side, quantity, order_type='MARKET'):
        # Simulate placing an order with Binance API
        endpoint = '/order'
        params = {
            'symbol': self.symbol,
            'side': side,
            'type': order_type,
            'quantity': quantity,
            'timestamp': int(time.time() * 1000)
        }
        
        query_string = '&'.join([f"{key}={params[key]}" for key in sorted(params)])
        signature = hmac.new(CONST.B_sKEY.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

        headers = {'X-MBX-APIKEY': CONST.B_KEY}
        
        query_string += f'&signature={signature}'

        response = requests.post(f"{self.base_url}{endpoint}?{query_string}", headers=headers)        
        return response.json()
    
    def place_buy_order_usdt(self, usdt_quantity, order_type='MARKET', price=None):
        # Get the current market price (replace 'BNBUSDT' with the actual symbol you are trading)
        ticker_endpoint = '/ticker/price'
        ticker_params = {'symbol': self.symbol}
        ticker_response = requests.get(f"{self.base_url}{ticker_endpoint}", params=ticker_params)
        current_price = float(ticker_response.json()['price'])

        # Calculate the quantity in the base asset (e.g., BNB)
        base_asset_quantity = usdt_quantity / current_price

        # Place a buy order using a signed request with the calculated quantity
        self.place_order('BUY', base_asset_quantity, order_type)
        
    def get_account_balances(self):
        # Retrieve account information including balances
        endpoint = '/account'
        params = {'timestamp': int(time.time() * 1000)}

        # Generate the signature
        query_string = '&'.join([f"{key}={params[key]}" for key in sorted(params)])
        signature = hmac.new(CONST.B_sKEY.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

        # Include both API key and signature in the headers
        headers = {'X-MBX-APIKEY': CONST.B_KEY}

        # Include the signature in the query string
        query_string += f'&signature={signature}'

        # Send the signed request to get account information
        response = requests.get(f"{self.base_url}{endpoint}?{query_string}", headers=headers)
        account_info = response.json()

        self.bnb_balance = float(next(filter(lambda x: x['asset'] == 'BNB', account_info['balances']))['free'])
        self.usdt_balance = float(next(filter(lambda x: x['asset'] == 'USDT', account_info['balances']))['free'])
        
        # Extract balances for specific assets (e.g., BNB and USDT)
        balances = {}
        for balance in account_info['balances']:
            asset = balance['asset']
            free_balance = float(balance['free'])
            balances[asset] = free_balance

        return balances
    
    def fetch_historical_data(self, limit=500):
        endpoint = '/klines'
        params = {'symbol': self.symbol, 'interval': self.interval, 'limit': limit}
        response = requests.get(f"{self.base_url}{endpoint}", params=params)
        data = response.json()

        # Create a DataFrame from the fetched data
        columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume',
                   'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']
        df_new = pd.DataFrame(data, columns=columns)
        df_new['timestamp'] = pd.to_datetime(df_new['timestamp'], unit='ms')

        # Concatenate new data with existing data
        self.df = pd.concat([self.df, df_new], ignore_index=True)
        
    def calculate_moving_averages(self):
        # Calculate moving averages
        self.df['MA_7'] = self.df['close'].rolling(window=7).mean()
        self.df['MA_25'] = self.df['close'].rolling(window=25).mean()
        self.df['MA_99'] = self.df['close'].rolling(window=99).mean()

In [47]:
bin_Acc = BinanceAccount()

In [ ]:
bin_Acc.place_buy_order_usdt(1, order_type='MARKET')